In [3]:
#Importing the pytorch libraries

import torch
import torchvision
from torchvision import transforms, datasets
from torch import optim
import torch.nn as nn
from torchsummary import summary
from torch.utils.data import DataLoader, random_split
device = torch.device('cpu')
from torch.utils.tensorboard import SummaryWriter

ImportError: cannot import name 'torchsummary' from 'torch' (D:\Python\envs\pytorch\lib\site-packages\torch\__init__.py)

In [2]:
#tensorboard dev upload --logdir {logs}

In [3]:
root_dir= r'C:\Users\BlueFlames\Python Projects\Datasets\Garbage classification\Garbage classification'
dataset = datasets.ImageFolder(root_dir, transform= transforms.Compose([transforms.RandomCrop(64),
                                                                    transforms.ToTensor()]))
#loader = DataLoader(dataset, batch_size=4, shuffle=True)
train_size = int(0.8*(len(dataset)))
test_size = int((len(dataset) - train_size)/2)
val_size = len(dataset) - train_size - test_size
train_data, test_data, val_data =  random_split(dataset, lengths=(train_size, test_size, val_size))

#Preparing train, test and validation datasets
train_loader= DataLoader(train_data, batch_size=20, shuffle= True)
test_loader= DataLoader(test_data, batch_size=20, shuffle=True)
val_loader = DataLoader(val_data, batch_size=20, shuffle= True)


In [4]:
(1520+202+202)
print(len(dataset))

2534


In [10]:
#Building a model

class GarbageNet(nn.Module):
    def __init__(self):
        super(GarbageNet,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3,stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.lf = nn.Linear(in_features=32*32*24, out_features=6)
     
        #Defining a forward pass 
    def forward(self, input):
        output = self.conv1(input)
        output = self.relu1(output)
        output = self.maxpool1(output)
        output = self.conv2(output)
        output = self.relu2(output)
        output = output.view(-1, 32 * 32 * 24)
        output = self.lf(output)
        return output


In [11]:
cnn_model = GarbageNet()

In [14]:
def trainNet(n_epochs):
#Backpropagation - Using SGD
    print("=======Hyperparameters=========")
    print("epochs=", n_epochs)
    
    optimizer = optim.SGD(cnn_model.parameters(), lr = 0.001, momentum = 0.9) 
#     optimizer  = optim.Adam(cnn_model.parameters())    #Defining an optimizer - SGD
    loss_fn = nn.CrossEntropyLoss()

    for epoch in range(n_epochs):
        cnn_model.train()
        for i,data in enumerate(train_loader):
            images, labels = data
            optimizer.zero_grad() #Forward pass, backward pass
            outputs = cnn_model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

In [15]:
trainNet(n_epochs=25)

=======Hyperparameters=========
epochs= 25


In [9]:
cnn_model.eval()

for k,(image, labels) in enumerate(test_data):
    output = cnn_model(image)
    _, prediction = torch.max(output,1)
    torch_acc_count += torch.sum(prediction == labels.data).item()
    test_accuracy = test_acc_count / len(test_dataset)

RuntimeError: Expected 4-dimensional input for 4-dimensional weight 12 3 3 3, but got 3-dimensional input of size [3, 64, 64] instead